In [15]:
import numpy as np
import re
import pandas as pd
import functools
from itertools import chain
from functools import partial
from pathlib import Path
import os

In [16]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/ML-Project/PII-DATA'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
train = pd.read_json("/content/drive/My Drive/ML-Project/PII-DATA/train.json")
test= pd.read_json('/content/drive/My Drive/ML-Project/PII-DATA/test.json')
traindf1 = pd.read_csv("/content/drive/My Drive/ML-Project/PII-DATA/pii_dataset.csv")

In [18]:
traindf1

,document,text,tokens,trailing_whitespace,labels,prompt,prompt_id,name,email,phone,job,address,username,url,hobby,len
0,1073d46f-2241-459b-ab01-851be8d26436,"My name is Aaliyah Popova, and I am a jeweler ...","['My', 'name', 'is', 'Aaliyah', 'Popova,', 'an...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUD...",\n Aaliyah Popova is a jeweler with 13 year...,1,Aaliyah Popova,aaliyah.popova4783@aol.edu,(95) 94215-7906,jeweler,97 Lincoln Street,NaN,NaN,Podcasting,363
1,5ec717a9-17ee-48cd-9d76-30ae256c9354,"My name is Konstantin Becker, and I'm a develo...","['My', 'name', 'is', 'Konstantin', 'Becker,', ...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUD...",\n Konstantin Becker is a developer with 2 ...,1,Konstantin Becker,konstantin.becker@gmail.com,0475 4429797,developer,826 Webster Street,NaN,NaN,Quilting,255
2,353da41e-7799-4071-ab20-d959b362612e,"As Mieko Mitsubishi, an account manager at a p...","['As', 'Mieko', 'Mitsubishi,', 'an', 'account'...","[True, True, True, True, True, True, True, Tru...","['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'O',...",\n Mieko Mitsubishi is a account manager. W...,3,Mieko Mitsubishi,mieko_mitsubishi@msn.org,+27 61 222 4762,account manager,1309 Southwest 71st Terrace,NaN,NaN,Metal detecting,259
3,9324ee01-7bdc-41b1-a7a5-01307f72c20d,"My name is Kazuo Sun, and I'm an air traffic c...","['My', 'name', 'is', 'Kazuo', 'Sun,', 'and', ""...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUD...",\n Kazuo Sun is a air traffic controller wi...,1,Kazuo Sun,kazuosun@hotmail.net,0304 2215930,air traffic controller,736 Sicard Street Southeast,NaN,NaN,Amateur radio,281
4,971fe266-2739-4f1b-979b-7f64e07d5a4a,"My name is Arina Sun, and I'm a dental hygieni...","['My', 'name', 'is', 'Arina', 'Sun,', 'and', ""...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUD...",\n Arina Sun is a dental hygienist. Write a...,3,Arina Sun,arina-sun@gmail.net,0412 1245924,dental hygienist,5701 North 67th Avenue,NaN,NaN,Related,210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,a90b6b71-0e77-4089-a3a4-9a5b5c5aec78,"Hello, I'm Nicholas Moore, a man with a rich t...","['Hello,', ""I'm"", 'Nicholas', 'Moore,', 'a', '...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUDENT',...",\n Write a fictional semi-formal biography ...,0,Nicholas Moore,nicholas_moore8047@yahoo.gov,+91-63720 22261,videographer,35915 Patrick Mews Suite 978,n.moore,https://www.nmoore.org,Surfing,360
4430,1492ed0e-f162-424f-9c40-f3edde790ca1,"Hello, my name is Alexey Novikov and I'm a psy...","['Hello,', 'my', 'name', 'is', 'Alexey', 'Novi...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME...",\n Alexey Novikov is a psychologist. Write ...,3,Alexey Novikov,alexey.novikov@msn.gov,0264 828 4342,psychologist,161 Creek Road,NaN,NaN,Gardening,248
4431,57ef34c1-48db-4413-9573-774021e57f63,"My name is Ludmila Inoue, and I'm a person wit...","['My', 'name', 'is', 'Ludmila', 'Inoue,', 'and...","[True, True, True, True, True, True, True, Tru...","['O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUD...",\n Write a fictional semi-formal biography ...,0,Ludmila Inoue,ludmila_inoue@outlook.net,(285) 815-7373,physician,706 Seagrove Road,NaN,NaN,Quilting,445
4432,a4486627-1c62-48b0-bf4f-53259ecc0a28,"Dr. Tu Garcia, a renowned dermatologist, embar...","['Dr.', 'Tu', 'Garcia,', 'a', 'renowned', 'der...","[True, True, True, True, True, True, True, Tru...","['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'O',...",\n Tu Garcia is a dermatologist. Write abou...,2,Tu Garcia,tugarcia@outlook.com,+91-14426 83047,dermatologist,1677 Anthony Run,t.garcia,http://blog.tu-garcia.edu,Skiing,320


In [19]:
train

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...,...
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [20]:
import pandas as pd

# Assuming 'df' is your pandas DataFrame and 'column_name' is the name of the column you're interested in
max_value = train['document'].max()

print(max_value)


22687


In [21]:
import ast

# Check the type of the first element in the 'tokens' and 'labels' columns
print(type(train['tokens'].iloc[0]))
print(type(train['labels'].iloc[0]))

# If the types are strings and you need to convert them to lists
if isinstance(train['tokens'].iloc[0], str):
    train['tokens'] = train['tokens'].apply(ast.literal_eval)
if isinstance(train['labels'].iloc[0], str):
    train['labels'] = train['labels'].apply(ast.literal_eval)


<class 'list'>
<class 'list'>


In [22]:
import ast

# Check the type of the first element in the 'tokens' and 'labels' columns
print(type(traindf1['tokens'].iloc[0]))
print(type(traindf1['labels'].iloc[0]))

# If the types are strings and you need to convert them to lists
if isinstance(traindf1['tokens'].iloc[0], str):
    traindf1['tokens'] = traindf1['tokens'].apply(ast.literal_eval)
if isinstance(traindf1['labels'].iloc[0], str):
    traindf1['labels'] = traindf1['labels'].apply(ast.literal_eval)

print(type(traindf1['tokens'].iloc[0]))
print(type(traindf1['labels'].iloc[0]))


<class 'str'>
<class 'str'>
<class 'list'>
<class 'list'>


In [23]:
train_data1 = pd.DataFrame({"document":traindf1["document"],"words":traindf1["tokens"],"trailing_whitespace":traindf1["trailing_whitespace"],"labels":traindf1['labels']})

In [24]:
train_data1

,document,words,trailing_whitespace,labels
0,1073d46f-2241-459b-ab01-851be8d26436,"[My, name, is, Aaliyah, Popova,, and, I, am, a...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
1,5ec717a9-17ee-48cd-9d76-30ae256c9354,"[My, name, is, Konstantin, Becker,, and, I'm, ...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
2,353da41e-7799-4071-ab20-d959b362612e,"[As, Mieko, Mitsubishi,, an, account, manager,...","[True, True, True, True, True, True, True, Tru...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."
3,9324ee01-7bdc-41b1-a7a5-01307f72c20d,"[My, name, is, Kazuo, Sun,, and, I'm, an, air,...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
4,971fe266-2739-4f1b-979b-7f64e07d5a4a,"[My, name, is, Arina, Sun,, and, I'm, a, denta...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
...,...,...,...,...
4429,a90b6b71-0e77-4089-a3a4-9a5b5c5aec78,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[True, True, True, True, True, True, True, Tru...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
4430,1492ed0e-f162-424f-9c40-f3edde790ca1,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
4431,57ef34c1-48db-4413-9573-774021e57f63,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
4432,a4486627-1c62-48b0-bf4f-53259ecc0a28,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[True, True, True, True, True, True, True, Tru...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [25]:
train


,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...,...
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [26]:

#building up train data and test data
train_data = pd.DataFrame({"document":train["document"],"words":train["tokens"],"trailing_whitespace":train["trailing_whitespace"],"labels":train['labels']})

In [27]:
train_data


,document,words,trailing_whitespace,labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...
6802,22678,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [28]:
train_data1

,document,words,trailing_whitespace,labels
0,1073d46f-2241-459b-ab01-851be8d26436,"[My, name, is, Aaliyah, Popova,, and, I, am, a...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
1,5ec717a9-17ee-48cd-9d76-30ae256c9354,"[My, name, is, Konstantin, Becker,, and, I'm, ...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
2,353da41e-7799-4071-ab20-d959b362612e,"[As, Mieko, Mitsubishi,, an, account, manager,...","[True, True, True, True, True, True, True, Tru...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."
3,9324ee01-7bdc-41b1-a7a5-01307f72c20d,"[My, name, is, Kazuo, Sun,, and, I'm, an, air,...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
4,971fe266-2739-4f1b-979b-7f64e07d5a4a,"[My, name, is, Arina, Sun,, and, I'm, a, denta...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
...,...,...,...,...
4429,a90b6b71-0e77-4089-a3a4-9a5b5c5aec78,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[True, True, True, True, True, True, True, Tru...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
4430,1492ed0e-f162-424f-9c40-f3edde790ca1,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
4431,57ef34c1-48db-4413-9573-774021e57f63,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
4432,a4486627-1c62-48b0-bf4f-53259ecc0a28,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[True, True, True, True, True, True, True, Tru...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [29]:
import pandas as pd

# Assuming the dataframe 'df' contains the data and the 'doc_id' column needs to be replaced with numbers starting from 1
train_data1['document'] = range(25000, len(train_data1) + 25000)

# Display the first few rows of the dataframe to confirm the change
train_data1

,document,words,trailing_whitespace,labels
0,25000,"[My, name, is, Aaliyah, Popova,, and, I, am, a...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
1,25001,"[My, name, is, Konstantin, Becker,, and, I'm, ...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
2,25002,"[As, Mieko, Mitsubishi,, an, account, manager,...","[True, True, True, True, True, True, True, Tru...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."
3,25003,"[My, name, is, Kazuo, Sun,, and, I'm, an, air,...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
4,25004,"[My, name, is, Arina, Sun,, and, I'm, a, denta...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
...,...,...,...,...
4429,29429,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[True, True, True, True, True, True, True, Tru...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
4430,29430,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
4431,29431,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
4432,29432,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[True, True, True, True, True, True, True, Tru...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [30]:
import pandas as pd

# Assuming result_df1 and result_df2 are your two dataframes that you want to join
result = pd.concat([train_data, train_data1], ignore_index=True)




In [31]:
result

,document,words,trailing_whitespace,labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...
11236,29429,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[True, True, True, True, True, True, True, Tru...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
11237,29430,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
11238,29431,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
11239,29432,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[True, True, True, True, True, True, True, Tru...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [32]:
result = result.drop('trailing_whitespace', axis=1)

In [34]:
result

,document,words,labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,"[Design, Thinking, for, Innovation, \n\n, Sind...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,"[Assignment, :, , Visualization, , Reflecti...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...
11236,29429,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
11237,29430,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
11238,29431,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
11239,29432,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [35]:
# # # Saving the 'results' DataFrame to a CSV file
# result.to_csv('/content/drive/My Drive/ML-Project/PII-DATA/token-labels.csv', index=False)  # Set index=False to avoid writing row indices


In [ ]:
# Modifying the script to print each word along with its corresponding label for labels that are not 'O'.

# Define the function to print words and their labels
def print_words_with_labels(words, labels):
    # Gather pairs of (word, label) where the label is not 'O'
    non_o_pairs = [(word, label) for word, label in zip(words, labels) if label != 'O']
    return non_o_pairs

# Iterate over each row in the DataFrame and print words with non-'O' labels
for index, row in result1.iterrows():
    non_o_pairs = print_words_with_labels(row['words'], row['labels'])
    if non_o_pairs:  # If there is at least one non-'O' label in the sentence
        print(f"Document {row['document']} Non-'O' word-label pairs:")
        for word, label in non_o_pairs:
            print(f"Word: {word} - Label: {label}")
        print("\n")  # Newline for readability between documents



Document 7 Non-'O' word-label pairs:
Word: Nathalie - Label: B-NAME_STUDENT
Word: Sylla - Label: I-NAME_STUDENT
Word: Nathalie - Label: B-NAME_STUDENT
Word: Sylla - Label: I-NAME_STUDENT
Word: Nathalie - Label: B-NAME_STUDENT
Word: Sylla - Label: I-NAME_STUDENT


Document 10 Non-'O' word-label pairs:
Word: Diego - Label: B-NAME_STUDENT
Word: Estrada - Label: I-NAME_STUDENT
Word: Diego - Label: B-NAME_STUDENT
Word: Estrada - Label: I-NAME_STUDENT




In [ ]:
# Function to check token-label alignment
def check_alignment(document_id, tokens, labels):
    if len(tokens) != len(labels):
        print(f"Alignment issue in document {document_id}:")
        print(f"Tokens ({len(tokens)}): {tokens}")
        print(f"Labels ({len(labels)}): {labels}")
        print("\n")
        return True
    return False

# Validate the reconstructed sentence and check alignment
alignment_issues = []
for index, row in result.iterrows():
    document_id = row['document']
    tokens = row['words']
    labels = row['labels']

    # Check alignment and if there is an issue, add it to the list
    if check_alignment(document_id, tokens, labels):
        alignment_issues.append(document_id)

# Print summary of documents with alignment issues
if alignment_issues:
    print(f"Found alignment issues in {len(alignment_issues)} documents: {alignment_issues}")
else:
    print("No alignment issues found.")


No alignment issues found.


In [ ]:
result

,document,words,labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,"[Design, Thinking, for, Innovation, \n\n, Sind...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,"[Assignment, :, , Visualization, , Reflecti...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...
11236,29429,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
11237,29430,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
11238,29431,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
11239,29432,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [ ]:
# def remove_break_tokens_and_labels(words, labels):
#     """Removes newline tokens ('\n' and '\n\n') and their corresponding labels."""
#     new_words = []
#     new_labels = []
#     for word, label in zip(words, labels):
#         # Skip the word and its label if the word is a newline or double newline token
#         if word not in ['\n', '\n\n']:
#             new_words.append(word)
#             new_labels.append(label)
#     return new_words, new_labels

# # Apply the updated function to all rows in the DataFrame 'result'
# result['words'], result['labels'] = zip(*result.apply(lambda row: remove_break_tokens_and_labels(row['words'], row['labels']), axis=1))

# result




def remove_break_tokens_and_labels(words, labels):
    """Removes newline tokens ('\n' and '\n\n'), empty tokens, and their corresponding labels."""
    new_words = []
    new_labels = []
    for word, label in zip(words, labels):
        # Skip the word and its label if the word is a newline, double newline, or empty token
        if word not in ['\n', '\n\n'] and word.strip():
            new_words.append(word)
            new_labels.append(label)
    return new_words, new_labels

# Apply the updated function to all rows in the DataFrame 'result'
result['words'], result['labels'] = zip(*result.apply(lambda row: remove_break_tokens_and_labels(row['words'], row['labels']), axis=1))

result

,document,words,labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,"[Diego, Estrada, Design, Thinking, Assignment,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,"[Reporting, process, by, Gilberto, Gamboa, Cha...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
3,20,"[Design, Thinking, for, Innovation, Sindy, Sam...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
4,56,"[Assignment, :, Visualization, Reflection, Sub...","[O, O, O, O, O, O, O, B-NAME_STUDENT, I-NAME_S..."
...,...,...,...
11236,29429,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
11237,29430,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
11238,29431,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
11239,29432,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [ ]:
# Modifying the script to print only the rows where the number of tokens does not match the number of labels.

def check_token_label_alignment(df):
    misaligned_rows = []
    for index, row in result.iterrows():
        words = row['words']
        labels = row['labels']
        # Check if the number of tokens and labels match
        if len(words) != len(labels):
            misaligned_rows.append((index, len(words), len(labels)))
    return misaligned_rows

# Assuming 'result' DataFrame is already loaded with the 'words' and 'labels' columns
misaligned_rows = check_token_label_alignment(result)

# Print only misaligned rows
for index, num_words, num_labels in misaligned_rows:
    print(f"Mismatch in row {index}: {num_words} tokens, {num_labels} labels.")



In [ ]:
# We will now write a script to print each token and its corresponding label for sentence 1 in the DataFrame.

# Get the data for sentence 1 (assuming the index starts at 0)
sentence_1_data = result.loc[6806]

# Extract the tokens and labels for sentence 1
tokens_sentence_1 = sentence_1_data['words']
labels_sentence_1 = sentence_1_data['labels']

# Print each token and its corresponding label for sentence 1
print(f"Tokens and labels for document {sentence_1_data['document']}:")
for token, label in zip(tokens_sentence_1, labels_sentence_1):
    print(f"Token: {token} - Label: {label}")





Tokens and labels for document 22687:
Token: Mind - Label: O
Token: Mapping - Label: O
Token: Challenge - Label: O
Token: My - Label: O
Token: consulting - Label: O
Token: teammates - Label: O
Token: and - Label: O
Token: I - Label: O
Token: , - Label: O
Token: specialized - Label: O
Token: in - Label: O
Token: analyzing - Label: O
Token: business - Label: O
Token: concerns - Label: O
Token: and - Label: O
Token: finding - Label: O
Token: the - Label: O
Token: best - Label: O
Token: solutions - Label: O
Token: for - Label: O
Token: their - Label: O
Token: problems - Label: O
Token: , - Label: O
Token: were - Label: O
Token: assigned - Label: O
Token: to - Label: O
Token: a - Label: O
Token: project - Label: O
Token: which - Label: O
Token: consisted - Label: O
Token: on - Label: O
Token: investigating - Label: O
Token: which - Label: O
Token: are - Label: O
Token: the - Label: O
Token: key - Label: O
Token: issues - Label: O
Token: that - Label: O
Token: determine - Label: O
Token: the

In [ ]:
result

,document,words,labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,"[Diego, Estrada, Design, Thinking, Assignment,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,"[Reporting, process, by, Gilberto, Gamboa, Cha...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
3,20,"[Design, Thinking, for, Innovation, Sindy, Sam...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
4,56,"[Assignment, :, Visualization, Reflection, Sub...","[O, O, O, O, O, O, O, B-NAME_STUDENT, I-NAME_S..."
...,...,...,...
11236,29429,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
11237,29430,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
11238,29431,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
11239,29432,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [ ]:
# Let's update the DataFrame 'result' by converting the list of words in the 'words' column
# into a sentence, then assign this back to the 'sentence' column of the DataFrame.

def words_to_sentence(words_list):
    # Join the list of words into a string, add a space between words
    return ' '.join(words_list)

# Apply the function to the 'words' column to create the 'sentence' column
result['sentence'] = result['words'].apply(words_to_sentence)

result


,document,words,labels,sentence
0,7,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...",Design Thinking for innovation reflexion - Avr...
1,10,"[Diego, Estrada, Design, Thinking, Assignment,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O...",Diego Estrada Design Thinking Assignment Visua...
2,16,"[Reporting, process, by, Gilberto, Gamboa, Cha...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O...",Reporting process by Gilberto Gamboa Challenge...
3,20,"[Design, Thinking, for, Innovation, Sindy, Sam...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O...",Design Thinking for Innovation Sindy Samaca Gi...
4,56,"[Assignment, :, Visualization, Reflection, Sub...","[O, O, O, O, O, O, O, B-NAME_STUDENT, I-NAME_S...",Assignment : Visualization Reflection Submitte...
...,...,...,...,...
11236,29429,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O...","Hello, I'm Nicholas Moore, a man with a rich t..."
11237,29430,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O...","Hello, my name is Alexey Novikov and I'm a psy..."
11238,29431,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O...","My name is Ludmila Inoue, and I'm a person wit..."
11239,29432,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O...","Dr. Tu Garcia, a renowned dermatologist, embar..."


In [ ]:
result.rename(columns={'labels': 'label_seq'}, inplace=True)
result.drop('document', axis=1, inplace=True)

In [ ]:
result

,words,label_seq,sentence
0,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...",Design Thinking for innovation reflexion - Avr...
1,"[Diego, Estrada, Design, Thinking, Assignment,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O...",Diego Estrada Design Thinking Assignment Visua...
2,"[Reporting, process, by, Gilberto, Gamboa, Cha...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O...",Reporting process by Gilberto Gamboa Challenge...
3,"[Design, Thinking, for, Innovation, Sindy, Sam...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O...",Design Thinking for Innovation Sindy Samaca Gi...
4,"[Assignment, :, Visualization, Reflection, Sub...","[O, O, O, O, O, O, O, B-NAME_STUDENT, I-NAME_S...",Assignment : Visualization Reflection Submitte...
...,...,...,...
11236,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O...","Hello, I'm Nicholas Moore, a man with a rich t..."
11237,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O...","Hello, my name is Alexey Novikov and I'm a psy..."
11238,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O...","My name is Ludmila Inoue, and I'm a person wit..."
11239,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O...","Dr. Tu Garcia, a renowned dermatologist, embar..."


In [ ]:
result = result.drop('words', axis=1)

In [ ]:
result =result[['sentence', 'label_seq']]

result

,sentence,label_seq
0,Design Thinking for innovation reflexion - Avr...,"[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,Diego Estrada Design Thinking Assignment Visua...,"[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,Reporting process by Gilberto Gamboa Challenge...,"[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
3,Design Thinking for Innovation Sindy Samaca Gi...,"[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
4,Assignment : Visualization Reflection Submitte...,"[O, O, O, O, O, O, O, B-NAME_STUDENT, I-NAME_S..."
...,...,...
11236,"Hello, I'm Nicholas Moore, a man with a rich t...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
11237,"Hello, my name is Alexey Novikov and I'm a psy...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
11238,"My name is Ludmila Inoue, and I'm a person wit...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
11239,"Dr. Tu Garcia, a renowned dermatologist, embar...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [ ]:
# Adjust the function to use the correct column name 'labels'
result['label_seq'] = result['label_seq'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Function to split sentences into words
def split_sentence(sentence):
    return sentence.split()

# Check if each word in each sentence has a corresponding label
for index, row in result.iterrows():
    words = split_sentence(row['sentence'])
    labels = row['label_seq']

    # Check if the number of words and labels match
    if len(words) != len(labels):
        print(f"Mismatch in row {index}: {len(words)} words, {len(labels)} labels.")
    else:
        break
        #print(f"Row {index} is correctly aligned.")



In [ ]:
final=result

In [ ]:
import pandas as pd

def validate_labels(sentence, label_seq):
    errors = []
    previous_label = None
    for word, label in zip(sentence.split(), label_seq):
        if label.startswith('I-') and previous_label not in {label, label.replace('I-', 'B-')}:
            errors.append(f"Incorrect 'I-' following '{previous_label}': {word} [{label}]")
        previous_label = label
    return errors

# Initialize a list to keep track of the indices of invalid rows
invalid_row_indices = []

# Validate the label sequences and collect invalid row indices
for index, row in final.iterrows():
    sentence, label_seq = row['sentence'], row['label_seq']
    if any(label != 'O' for label in label_seq):
        errors = validate_labels(sentence, label_seq)
        if errors:
            invalid_row_indices.append(index)
            print(f"Sentence {index}: '{sentence}'")
            for error in errors:
                print(f"  Error: {error}")

# Count the number of invalid rows
num_invalid_rows = len(invalid_row_indices)
print(f"Number of invalid rows: {num_invalid_rows}")

# Delete the invalid rows from the DataFrame
final = final.drop(invalid_row_indices)

# Reset the index of the DataFrame if needed
final.reset_index(drop=True, inplace=True)


Sentence 6966: 'My name is Irina Martin, and I'm a fashion designer with 15 years of experience in the fashion industry. I have a passion for creating unique and stylish clothing that empowers women to feel confident and beautiful. I live at 5 Irving Street and can be reached at 0305 606 4179 or by email at irina.martin2345@yahoo.gov. One of my most memorable projects was designing a collection for a major fashion show in New York City. The theme of the show was "Empowerment," and I wanted to create pieces that reflected strength, femininity, and individuality. I spent weeks sketching out designs and selecting fabrics, and I collaborated with a team of talented seamstresses to bring my vision to life. The night of the show, I couldn't believe how stunning the collection looked on the runway. The models exuded confidence as they strutted down the catwalk, and the audience responded with thunderous applause. It was an incredible feeling to see my designs come to life and to witness the i

In [ ]:
final

,sentence,label_seq
0,Design Thinking for innovation reflexion - Avr...,"[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,Diego Estrada Design Thinking Assignment Visua...,"[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,Reporting process by Gilberto Gamboa Challenge...,"[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
3,Design Thinking for Innovation Sindy Samaca Gi...,"[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
4,Assignment : Visualization Reflection Submitte...,"[O, O, O, O, O, O, O, B-NAME_STUDENT, I-NAME_S..."
...,...,...
11210,"Hello, I'm Nicholas Moore, a man with a rich t...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
11211,"Hello, my name is Alexey Novikov and I'm a psy...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
11212,"My name is Ludmila Inoue, and I'm a person wit...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
11213,"Dr. Tu Garcia, a renowned dermatologist, embar...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [ ]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11215 entries, 0 to 11214
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentence   11215 non-null  object
 1   label_seq  11215 non-null  object
dtypes: object(2)
memory usage: 175.4+ KB


In [ ]:
new_df = pd.DataFrame(columns=['document', 'token', 'label'])

# Use pandas.concat for more efficiency
data_frames = []  # To collect all the small dataframes

# Process each row in the DataFrame
for index, row in final.iterrows():
    # Tokenize the sentence
    tokens = row['sentence'].split()
    # Get the label sequence
    labels = row['label_seq']

    # Make sure the length of tokens and labels are the same
    min_length = min(len(tokens), len(labels))
    tokens = tokens[:min_length]
    labels = labels[:min_length]

    # Create a small dataframe for the current row and add it to the list
    temp_df = pd.DataFrame({
        'document': [index] * min_length,
        'token': tokens,
        'label': labels
    })
    data_frames.append(temp_df)

# Concatenate all small dataframes into one
new_df = pd.concat(data_frames, ignore_index=True)




In [ ]:
new_df

,document,token,label
0,0,Design,O
1,0,Thinking,O
2,0,for,O
3,0,innovation,O
4,0,reflexion,O
...,...,...,...
5972025,11214,overall,O
5972026,11214,success,O
5972027,11214,of,O
5972028,11214,the,O


In [ ]:

# # Saving the 'results' DataFrame to a CSV file
new_df.to_csv('/content/drive/My Drive/ML-Project/PII-DATA/final.csv', index=False)  # Set index=False to avoid writing row indices
